### Introducción
<center><img src="https://mlflow.org/docs/0.4.1/_static/MLflow-logo-final-black.png" width="300" align="center" /><center>
 <center><h1><i>“platform for the machine learning lifecycle”</i></h1><center>
MLFlow es una plataforma open source que permite administrar el ciclo de vida de ML, incluyendo la experimentación, reproducibilidad y deploy de modelos, además de brindar la posibilidad de comparar la performance de los modelos obtenidos a lo largo del proyecto. Es posible utilizarla con múltiples lenguajes, tanto de manera local como en la nube.

## Ejemplo
#### Iris DataSet <br>
Dataset = Un conjunto de datos o dataset corresponde a los contenidos de una única tabla de base de datos o una única matriz de datos de estadística, donde cada columna de la tabla representa una variable en particular, y cada fila representa a un miembro determinado del conjunto de datos que estamos tratando.<br>
Dataset Iris = Contiene datos para cuantificar la variación morfológica de la flor Iris de tres especies relacionadas. <br>
Columnas:
* Largo de sépalo
* Ancho de sépalo
* Largo de pétalo
* Ancho de pétalo
* Especies <- lo que queremos predecir


<center><img src="https://www.w3resource.com/w3r_images/iris_flower_dataset.png" width="600" align="center" /><center>

<center><img src="https://miro.medium.com/max/1100/0*SHhnoaaIm36pc1bd" width="700" align="center" /><center>
<br>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Código sin tracking

In [ ]:
!pip install mlflow --quiet

#### Importar librerias

In [ ]:
import numpy as np
import pandas as pd # procesamiento de datos
import mlflow
import mlflow.sklearn # como voy a entrenar un modelo de sklearn, debo importar este módulo de mlflow
from sklearn import datasets # para importar el dataset iris directamente desde los dataset que provee sklearn

#### Cargar Dataset

In [ ]:
iris_X, iris_y = datasets.load_iris(return_X_y=True)
np.unique(iris_y)
np.random.seed(0)
indices = np.random.permutation(len(iris_X))
iris_X_train = iris_X[indices[:-10]]
iris_y_train = iris_y[indices[:-10]]
iris_X_test = iris_X[indices[-10:]]
iris_y_test = iris_y[indices[-10:]]

#### Entrenar modelo y predecir

In [ ]:
# Create and fit a nearest-neighbor classifier
from sklearn.neighbors import KNeighborsClassifier

n = 5

knc = KNeighborsClassifier(n_neighbors = n)
knc.fit(iris_X_train, iris_y_train)

knc.predict(iris_X_test)
score = knc.score(iris_X_test, iris_y_test)

print('Metrica: {}'.format(score))
print('Predicción: {}'.format(knc.predict(iris_X_test)))
print('Valor Real: {}'.format(iris_y_test))

Metrica: 0.9
Predicción: [1 2 1 0 0 0 2 1 2 0]
Valor Real: [1 1 1 0 0 0 2 1 2 0]


In [ ]:
iris_X_test

array([[5.6, 3. , 4.1, 1.3],
       [5.9, 3.2, 4.8, 1.8],
       [6.3, 2.3, 4.4, 1.3],
       [5.5, 3.5, 1.3, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.9, 3.1, 1.5, 0.1],
       [6.3, 2.9, 5.6, 1.8],
       [5.8, 2.7, 4.1, 1. ],
       [7.7, 3.8, 6.7, 2.2],
       [4.6, 3.2, 1.4, 0.2]])

### Incorporar MLFlow

**Sentencias a utilizar:**
* mlflow.start_run()
* mlflow.log_param()
* mlflow.log_params()
* mlflow.log_metric()
* mlflow.log_metrics()
* mlflow.sklearn.log_model()
* mlflow.end_run()

##### Crear/Setear experimento

In [ ]:
# 1- Crear experimento en el mismo lugar donde tiene su notebook con el nombre 'classifier-iris' puede ser mediante la interfaz, o utilizando código '''mlflow.create_experiment('path')'''
path = 'file:/content/drive/My Drive/Activity/mlruns/'
name = 'classifier-iris'
try:
  mlflow.set_tracking_uri(path)
  mlflow.create_experiment(name)
except:
  print('Experimiento ya creado')

Experimiento ya creado


In [ ]:
# 2- Setear experimento a usar
mlflow.set_experiment(name)

<Experiment: artifact_location='file:///content/drive/My Drive/WiDS-MLFlow/Demo/907257242944673571', creation_time=1685378902516, experiment_id='907257242944673571', last_update_time=1685378902516, lifecycle_stage='active', name='classifier-iris', tags={}>

##### Empezar trackeo de una ejecución de experimeintación

In [ ]:
# Arrancar un run
mlflow.start_run()

<ActiveRun: >

##### Registrar parámetros, métricas y modelos

In [ ]:
# Create and fit a Random Forest classifier
from sklearn.ensemble import RandomForestClassifier

n_arboles = 50
profundidad = 10

#------Loguear parámetros utilizando variables existentes-------
# En este segmento registrar los dos parámetros 'n_arboles' y 'profundidad' que hacen referencias a las variables del mismo nombre, utilizando *log_param*
mlflow.log_param('n_arboles', n_arboles)
mlflow.log_param('profundidad', profundidad)
#-------------------------------------------------------------

rf = RandomForestClassifier(n_estimators = n_arboles, max_depth = profundidad)
rf.fit(iris_X_train,iris_y_train)

#---------Loguear un parámetro creando valores ---------------
# En este segmento registrar un parámetro llamado '_tipo_algoritmo', donde el valor sea 'RandomForestClassifier', utilizando *log_param*
mlflow.log_param('_tipo_algoritmo','RandomForestClassifier')
#-------------------------------------------------------------

#---------Loguear parámetros usando un diccionario -----------
# En este segmento registrar un  diccionario obtenido a partir de ejecutar *rf.get_params()*  con el nombre de '_tipo_algoritmo', utilizando *log_params*
mlflow.log_params(rf.get_params())
#-------------------------------------------------------------


score = rf.score(iris_X_test, iris_y_test)
#---------------------Loguear métricas -----------------------
# En este segmento registrar el score llamado 'score_iris', donde el valor sea la variable score creada en el paso anterior, utilizando *log_metric*
mlflow.log_metric('score_iris', score)
#-------------------------------------------------------------

#---------------------Loguear modelo--------------------------
# En este segmento registrar el modelo con el nombre 'model', donde el objeto a guardar sea 'rf', utilizando *log_model*
mlflow.sklearn.log_model(rf, 'model')
#-------------------------------------------------------------

print('Metrica: {}'.format(score))
print('Predicción: {}'.format(rf.predict(iris_X_test)))
print('Valor Real: {}'.format(iris_y_test))

Metrica: 0.9
Predicción: [1 2 1 0 0 0 2 1 2 0]
Valor Real: [1 1 1 0 0 0 2 1 2 0]


##### Terminar trackeo de la ejecución

In [ ]:
# Finalizar la ejecución
mlflow.end_run()

##### Nueva prueba de ejecución usando with statement

In [ ]:
# Reutilizando el código anterior con la implementación de MLFlow,
# pero en vez de usar mlflow.start_run() y mlflow.end_run() utilizar  *with mlflow.start_run()*
# y aplicando un modelo diferente

with mlflow.start_run():
    # Create and fit a nearest-neighbor classifier
    from sklearn.neighbors import KNeighborsClassifier

    n = 5

    #mlflow.log_param('n_neighbors', n) # Guardar parametro "n" correspondiente al hyperparametro n_neighbors de los KNN

    knc = KNeighborsClassifier(n_neighbors = n)
    knc.fit(iris_X_train, iris_y_train)

    mlflow.log_param('_tipo_algoritmo','KNeighborsClassifier')
    mlflow.log_params(knc.get_params())

    knc.predict(iris_X_test)
    score = knc.score(iris_X_test, iris_y_test)

    mlflow.log_metric('score_iris', score) # Guardar la métrica score obtenido de knn
    mlflow.sklearn.log_model(knc, 'model') # Guardar la métrica score obtenido de knn

    print('Metrica: {}'.format(score))
    print('Predicción: {}'.format(knc.predict(iris_X_test)))
    print('Valor Real: {}'.format(iris_y_test))

Metrica: 0.9
Predicción: [1 2 1 0 0 0 2 1 2 0]
Valor Real: [1 1 1 0 0 0 2 1 2 0]


### Explorar los resultados obtenidos
Podemos ver los resultados que fuimos guardando en diferentes corridas a partir del comando `search_runs`.
Primero podemos consultar los experimentos disponibles usando el comando: `mlflow.tracking.MlflowClient().list_experiments()`

In [ ]:
experimentos = mlflow.tracking.MlflowClient().search_experiments()
experimentos

[<Experiment: artifact_location='file:///content/drive/My Drive/WiDS-MLFlow/Demo/810590226620531637', creation_time=1685378923868, experiment_id='810590226620531637', last_update_time=1685378923868, lifecycle_stage='active', name='file:/content/drive/My Drive/WiDS-MLFlow/Demo/', tags={}>,
 <Experiment: artifact_location='file:///content/drive/My Drive/WiDS-MLFlow/Demo/907257242944673571', creation_time=1685378902516, experiment_id='907257242944673571', last_update_time=1685378902516, lifecycle_stage='active', name='classifier-iris', tags={}>,
 <Experiment: artifact_location='file:///content/drive/My Drive/WiDS-MLFlow/Demo/0', creation_time=1685378902497, experiment_id='0', last_update_time=1685378902497, lifecycle_stage='active', name='Default', tags={}>]

In [ ]:
df = pd.DataFrame()
for exp in experimentos:
  row = [exp.artifact_location, exp.experiment_id, exp.lifecycle_stage, exp.name, exp.tags]
  df = df.append([row])
df.columns = ['artifact_location','experiment_id','lifecycle_stage','name','tags']
df

<ipython-input-33-86d33fd38b68>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([row])


,artifact_location,experiment_id,lifecycle_stage,name,tags
0,file:///content/drive/My Drive/WiDS-MLFlow/Dem...,810590226620531637,active,file:/content/drive/My Drive/WiDS-MLFlow/Demo/,{}
0,file:///content/drive/My Drive/WiDS-MLFlow/Dem...,907257242944673571,active,classifier-iris,{}
0,file:///content/drive/My Drive/WiDS-MLFlow/Demo/0,0,active,Default,{}


In [ ]:
mlflow.search_runs().head(5)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.score_iris,params.metric,params.p,params._tipo_algoritmo,...,params.ccp_alpha,params.max_leaf_nodes,params.random_state,params.min_samples_split,params.max_features,tags.mlflow.runName,tags.mlflow.log-model.history,tags.mlflow.source.type,tags.mlflow.source.name,tags.mlflow.user
0,79e9ced946c146fc9a144f6e46d39dff,714131908109212751,FINISHED,file:///content/mlruns/714131908109212751/79e9...,2023-05-29 16:45:26.166000+00:00,2023-05-29 16:45:28.422000+00:00,0.9,minkowski,2,KNeighborsClassifier,...,None,None,None,None,None,sincere-shoat-396,"[{""run_id"": ""79e9ced946c146fc9a144f6e46d39dff""...",LOCAL,/usr/local/lib/python3.10/dist-packages/ipyker...,root
1,4e8d2fca786d41b5853bb49cf75c39ee,714131908109212751,FINISHED,file:///content/mlruns/714131908109212751/4e8d...,2023-05-29 16:45:22.051000+00:00,2023-05-29 16:45:26.146000+00:00,0.9,None,None,RandomForestClassifier,...,0.0,None,None,2,sqrt,welcoming-fly-807,"[{""run_id"": ""4e8d2fca786d41b5853bb49cf75c39ee""...",LOCAL,/usr/local/lib/python3.10/dist-packages/ipyker...,root


### Formas de filtrar los runs
Hay dos clases para los comparadores: numericos y string.<br>

__Comparadores númericos__  (metrics): =, !=, >, >=, <, and <=.<br>
__Comparadores para string__  (params, tags, and attributes): = and !=.<br>

##### Ejemplos:
Para buscar el subconjunto de ejecuciones con una métrica de precisión registrada mayor que 0.92:<br>
`metrics.accuracy > 0.92`

Para buscar ejecuciones creadas utilizando un modelo de Regresión logística, una tasa de aprendizaje (lambda) de 0.001 y una métrica de error registrada por debajo de 0.05:<br>
`params.model = "LogisticRegression" and params.lambda = "0.001" and metrics.error <= 0.05`

Para buscar los runs que han fallado en su ejecución:<br>
`attributes.status = "FAILED"`

In [ ]:
mlflow.search_runs(experiment_ids='753739227857488806', filter_string= 'metrics.score_iris > 0.7').head()

,run_id,experiment_id,status,artifact_uri,start_time,end_time


### Cargar un modelo guardado

In [ ]:
type(mlflow.search_runs())

pandas.core.frame.DataFrame

Obtener direccion del run del cual nos interesa recuperar el modelo guardado

In [ ]:
model_uri = mlflow.search_runs()\
        [mlflow.search_runs()['run_id'] == '8a813d8576c541759c676afb9eae2424']\
        .artifact_uri.item()
model_uri

ValueError: ignored

In [ ]:
model = mlflow.sklearn.load_model(model_uri+'/model')

In [ ]:
type(model)

In [ ]:
model= mlflow.sklearn.load_model(model_uri+'/model')
print('Predicción: {}'.format(model.predict(iris_X_test)))
print('Valor Real: {}'.format(iris_y_test))

In [ ]:
df = pd.DataFrame([model.predict(iris_X_test),iris_y_test]).transpose().rename({0:"predicho", 1:"valor_real"}, axis = 1)
iris_class = {0:'Iris-Setosa',1:'Iris-Versicolour',2:'Iris-Virginica'}
df.replace(iris_class, inplace = True)
df['son_iguales'] = df.apply(lambda x: True if x.predicho == x.valor_real else False, axis = 1)

In [ ]:
df

### Interfaz
Tambien contamos con una interfaz gráfica para el analisis de los resultados de los modelo.
Podemos acceder escribiendo en la consola de anaconda **mlflow ui** parados en el directorio donde se encuentran almacenados los runs de mlflow. Podemos ver la interfaz accediendo a http://localhost:5000/ en nuestro navegador.
<center><img src="https://miro.medium.com/max/4872/1*Swexh591ukDYIWNhTQ7YNA.png" width="800" align="center" /><center>